In [1]:
import pandas as pd
import glob
import numpy as np
import plotly.io as pio
import plotly.graph_objects as go
import plotly.express as px
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
from datetime import datetime
from dateutil import tz, parser
import warnings
warnings.filterwarnings('ignore')
import os

In [2]:
# path = '../Data/Analysis' # use your path
path = r'C:\Users\umama\shravya python assignment\Analysis'
all_files = glob.glob(path + "/*.csv")
all_files_dict={}

for csv in all_files:
    meter_name=os.path.basename(csv).replace('_results.csv','')
    all_files_dict[meter_name]=os.path.abspath(csv)


In [3]:
# labels = pd.read_csv('../data/Meter Names and Labels.csv')
labels = pd.read_csv(r'C:\Users\umama\shravya python assignment\Meter Names and Labels.csv')
labels = labels.rename(columns={"Name": "Building_Name"})
labels['Building_Name'] = labels['Building_Name'].str.replace("'" , " ")
labels['Building_Name'] = labels['Building_Name'].str.replace('-' , "")
labels['Building_Name'] = labels['Building_Name'].str.replace('"Spencer Hall"' , '')
labels['Building_Name'] = labels['Building_Name'].str.replace('_kWh' , "")
labels['Building_Name'] = labels['Building_Name'].str.strip()
labels
# print(labels.Building_Name.unique())

,Building_Name,Label
0,Baseball,Baseball Stadium (177) - Main Meter
1,BryanBldg,Bryan Building (082) - Main Meter
2,CampusSupplyStore,Campus Supply Store (050) - Main Meter
3,CarmichaelBldg,Carmichael Building (032) - Main Meter
4,ChemicalSafety,Chemical Safety Facility (174) - Main Meter
...,...,...
76,StudentRecCenterBasement,SRC Basement
77,StudentRecSiteLts222,SRC Site Lights 222
78,TennisCourts,Tennis Courts
79,UnderpassNorth,Underpass North


In [4]:
# find building names
building_options=[]
build_name = {}
print(labels)
for index, row in labels.iterrows():
#     print(row['c1'], row['c2'])
    building_options.append({'label':row['Label'],'value':row['Building_Name']})
print(building_options)
# for building_name in list(labels.Label.unique()):
#     building_options.append({'label':building_name,'value':building_name})
   
# year options
year_options = []
years = [2015,2016,2017,2018,2019,2020]
for year in years:
    year_options.append({'label':year,'value':year})


               Building_Name  \
0                   Baseball   
1                  BryanBldg   
2          CampusSupplyStore   
3             CarmichaelBldg   
4             ChemicalSafety   
..                       ...   
76  StudentRecCenterBasement   
77      StudentRecSiteLts222   
78              TennisCourts   
79            UnderpassNorth   
80        WalkerDeckLighting   

                                                Label  
0                 Baseball Stadium (177) - Main Meter  
1                   Bryan Building (082) - Main Meter  
2              Campus Supply Store (050) - Main Meter  
3              Carmichael Building (032) - Main Meter  
4        Chemical Safety Facility (174) - Main Meter   
..                                                ...  
76                                       SRC Basement  
77                                SRC Site Lights 222  
78                                      Tennis Courts  
79                                   Underpass North   

In [5]:
app = dash.Dash(__name__)
# App layout
app.layout = html.Div([
    
    html.H1("UNCG ENERGY CONSUMPTION DASHBOARD", style={'text-align': 'center'}),
    
    html.Label("CONSUMPTION", style={'text-align': 'center'}),
    
    dcc.RadioItems(id='consumption',
        options=[
            {'label': 'Total', 'value': 'Total'},
            {'label': 'Average', 'value': 'Average'}
        ],
        value='Total'
    ),
    
    html.Label("Frequency", style={'text-align': 'center'}),
    
    dcc.RadioItems(id='freq',
        options=[
            {'label': 'Hourly', 'value': 'hourly'},
            {'label': 'Daily',  'value': 'daily'},
            {'label': 'Weekly', 'value': 'weekly'},
            {'label': 'Monthly','value': 'monthly'}
        ],
        value='monthly'
    ),
    
    dcc.Checklist(id='check',
    options=[
        {'label': 'Actual', 'value': 'Actual'},
        {'label': 'Predicted', 'value': 'Predicted'}
    ],
    value=['Actual']),
    
    dcc.Dropdown(
        id='meters',
        options=building_options,
        value=['Baseball'],
        multi=True),
    ## Added Change1   ######
    html.Label("Year", style={'text-align': 'center'}),
    ###### Added end #######
    dcc.Checklist(id='years',
    options=year_options,
    value=[2020],
    labelStyle={'display': 'inline-block'}), 
    
    html.Div(id='dd-output-container'),
    
    html.Div(id='output_container', children=[]),
    
    html.Br(),

    dcc.Graph(id='predicted_graph', figure={})

])


In [6]:
# Connect the Plotly graphs with Dash Components
@app.callback(
    [Output(component_id='output_container', component_property='children'),
     Output(component_id='predicted_graph', component_property='figure')],
    [Input(component_id='consumption', component_property='value'),
    Input(component_id='freq', component_property='value'),
    Input(component_id='check', component_property='value'),
    Input(component_id='meters', component_property='value'),
    Input(component_id='years', component_property='value')]
)
def update_graph(cons_agg,freq,act_pred,meters,years):
    print('Chosen values are:',cons_agg,freq,act_pred,meters,years)

    container = "The chosen values by user are: {}".format(freq)


    def get_data(cons_agg,freq,act_pred,meter,years):
        
        cols = ['Actual','Predicted','obs_ci_lower','obs_ci_upper','Datetime']
        file_path=all_files_dict[meter]
        csv_file = open(file_path)
        dff = pd.read_csv(csv_file ,encoding = "ISO-8859-1",engine = 'python', sep=',', header=0,names=cols,error_bad_lines=False)
        dff['Datetime'] = pd.to_datetime(dff['Datetime'],utc=True,infer_datetime_format=False)
        dff_sel_year = dff[dff.Datetime.dt.year.isin(years) ]
#         print(dff_sel_year.head())
        
        if freq=='daily':
            if cons_agg == 'Total':
                dff1 = dff_sel_year.groupby(pd.Grouper(key='Datetime', freq='D'))['Actual','Predicted','obs_ci_upper','obs_ci_lower'].sum().reset_index()
            else:
                dff1 = dff_sel_year.groupby(pd.Grouper(key='Datetime', freq='D'))['Actual','Predicted','obs_ci_upper','obs_ci_lower'].mean().reset_index()

        elif freq=='weekly':
            if cons_agg == 'Total':
                dff1 = dff_sel_year.groupby(pd.Grouper(key='Datetime', freq='W'))['Actual','Predicted','obs_ci_upper','obs_ci_lower'].sum().reset_index()
            else:
                dff1 = dff_sel_year.groupby(pd.Grouper(key='Datetime', freq='W'))['Actual','Predicted','obs_ci_upper','obs_ci_lower'].mean().reset_index()
        elif freq=='monthly':
            if cons_agg == 'Total':
                dff1=dff_sel_year.groupby(pd.Grouper(key='Datetime',freq='M'))['Actual','Predicted','obs_ci_upper','obs_ci_lower'].sum().reset_index()
            else:
                dff1 = dff_sel_year.groupby(pd.Grouper(key='Datetime', freq='M'))['Actual','Predicted','obs_ci_upper','obs_ci_lower'].mean().reset_index()
        else:
            # Hourly
            if cons_agg == 'Total':
                dff1 = dff_sel_year
            else:
                dff1 = dff_sel_year
        csv_file.close()
        return(dff1)

    
    if type(meters)==str:
        meters=[meters]
    fig = go.Figure()   
    for meter in meters:
        
        data=get_data(cons_agg,freq,act_pred,meter,years)
        if 'Actual' in act_pred:     
#             fig = px.scatter(x=data['Datetime'], y=data['Actual'],)
            fig.add_trace(
                go.Scatter(x=data['Datetime'],
                            y=data['Actual'], 
                            mode='lines+markers', line={'dash': 'solid'},
                            name='Actual-'+meter)
            )
        if 'Predicted' in act_pred:
            fig.add_trace(go.Scatter(x=data['Datetime'], y=data['Predicted'],
                                mode='lines+markers',
                                line={'dash': 'dash'},
                                name='Predicted-'+meter))
            fig.add_trace(go.Scatter(
                                name='Upper Bound',
                                x=data['Datetime'],
                                y=data['obs_ci_upper'],
                                mode='lines',
                                marker=dict(color="#444"),
                                line=dict(width=0),
                                showlegend=False))
            fig.add_trace(go.Scatter(
                                name='Lower Bound',
                                x=data['Datetime'],
                                y=data['obs_ci_lower'],
                                marker=dict(color="#444"),
                                line=dict(width=0),
                                mode='lines',
                                fillcolor='rgba(68, 68, 68, 0.3)',
                                fill='tonexty',
                                showlegend=False))
# marker=dict(size=7, color="red") 
    ###### modified Change2 start ##########
    a = ''
    
    if freq=='hourly':
        a = 'Hour'
        xax = 'Time of day'
    elif freq=='daily':
        a = 'Day'
        xax = 'Date'
    elif freq=='weekly':
        a = 'Week'
        xax = 'Week'
    elif freq=='monthly':
        a = 'Month'
        xax = 'Month, Year'
    if cons_agg = 'Average':
        title = "Average Hourly Consumption at UNCG per "+ a
    else cons_agg = 'Total':
        title = "Total Consumption at UNCG per "+ a
    fig.update_layout(
        title_text=title,
        title_xanchor="center",
        title_font=dict(size=24),
        title_x=0.5,
        showlegend=True
    )
    

    fig.update_xaxes(title_text=xax)
    fig.update_yaxes(title_text=cons_agg+' Energy consumption (KWH)')
    fig.update_layout(hovermode="x unified")
    ############ modified Change2 end#####
    return container, fig



In [ ]:
if __name__ == '__main__':
    #app.run_server(debug=True)
    app.run_server(debug=False,host = '127.0.0.1')
    
    
    

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Nov/2020 03:47:03] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2020 03:47:03] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2020 03:47:03] "GET /_dash-dependencies HTTP/1.1" 200 -


Chosen values are: Total monthly ['Actual'] ['Baseball'] [2020]


127.0.0.1 - - [22/Nov/2020 03:47:05] "POST /_dash-update-component HTTP/1.1" 200 -


Chosen values are: Total monthly ['Actual', 'Predicted'] ['Baseball'] [2020]


...

127.0.0.1 - - [22/Nov/2020 13:46:49] "POST /_dash-update-component HTTP/1.1" 200 -


Chosen values are: Average weekly ['Actual'] ['Baseball'] [2020]


127.0.0.1 - - [22/Nov/2020 13:47:32] "POST /_dash-update-component HTTP/1.1" 200 -


Chosen values are: Average monthly ['Actual'] ['Baseball'] [2020]


127.0.0.1 - - [22/Nov/2020 13:47:48] "POST /_dash-update-component HTTP/1.1" 200 -
